# Lab 3

### Team Members:
 - Alex Lopez
 - Chris Haub
 - Erin McClure-Price
 - Chad Kwong

#### Library Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
#import turicreate as tc
from sklearn.preprocessing import StandardScaler
import plotly.express as px #EMP used for some outlier stuff
from scipy import stats #EMP used for some outlier stuff

ModuleNotFoundError: No module named 'plotly'

#### Data Import

In [ ]:
%%time

#Loading all data file separately 
vle = pd.read_csv('./anonymiseddata/vle.csv')
assessments = pd.read_csv('./anonymiseddata/assessments.csv')
courses = pd.read_csv('./anonymiseddata/courses.csv')
studentAssessments = pd.read_csv('./anonymiseddata/studentAssessment.csv')
studentInfo = pd.read_csv('./anonymiseddata/studentInfo.csv')
studentRegistration = pd.read_csv('./anonymiseddata/studentRegistration.csv')
studentVle = pd.read_csv('./anonymiseddata/studentVle.csv')

# Business Understanding 1 #
*Describe the purpose of the data set you selected (i.e., why was this data collected in the first place?). How will you measure the effectiveness of a good algorithm? Why does your chosen validation method make sense for this specific dataset and the stakeholders needs?*

For Lab 3 we chose to use the "Open University Learning Analytics Dataset" (OULAD) which is an anonymized collection of information acquired from online students who were enrolled in the Open University (Milton Keynes, England) in 2013-2014. The data includes student demographic and registration information, as well as assessment results and interactions with the Virtual Learning Environment (VLE). There are multiple ways that the OULAD can be utilized, including prediction of grade results, better understanding of the factors that influence online student outcomes, or to compare online student results to in-person student results. The reason why this dataset is important is because it can be used by educators and universities to increase student success within an online learning environment. 

Our goal for Lab 3 was to explore the OULAD using K-Means clustering in order to find trends and insight into student learning styles. We chose K-Means clustering because it is an unsupervised algorithm that is straightforward to carry out, scalable, and can be used for large datasets. We chose to evaluate the clustering model using the **Dunn Index..?**, which calculates the closest distance between two clusters divided by the largest distance between two clusers. The Dunn Index has a range of 0 to 1, wherein the closer the value is to 1 the better the clustering. We felt that the Dunn Index was ideal because it provides a numeric value to the fitness of a cluster rather than through visual analysis.


##### References
[Kaggle: OULAD](https://www.kaggle.com/datasets/rocki37/open-university-learning-analytics-dataset)

[K-Means Clustering and Dunn Index Implementaion 
From Scratch]( https://mayankdw.medium.com/k-means-clustering-and-dunn-index-implementaion-from-scratch-9c66573bfe90)

[Dunn Index for K-Means Clustering Evaluation](https://python-bloggers.com/2022/03/dunn-index-for-k-means-clustering-evaluation/)

[A Simple Explanation of K-Means Clustering](https://www.analyticsvidhya.com/blog/2020/10/a-simple-explanation-of-k-means-clustering/)

# Data Understanding 1 #
*Describe the meaning and type of data (scale, values, etc.) for each attribute in the data file. Verify data quality: Are there missing values? Duplicate data? Outliers? Are those mistakes? How do you deal with these problems?*

The OULAD contains seven CSV files that encompasses data collected from 32,593 students that were enrolled in 22 courses. There are a total of 43 variables and over **X** million instances. The CSV files and their contents are described below, information regarding each variable was taken from [HERE](https://analyse.kmi.open.ac.uk/open_dataset). It should be noted that "modules" refer to university courses and VLE refers to "virtual learning environment".

#### Courses:
* code_module **(categorical)** - code for the module, which serves as the identifier
* code_presentation  **(categorical)**– code name of the presentation. It consists of the year and “B” for the presentation starting in February and “J” for the presentation starting in October.
* length **(numeric)** - length of the module-presentation in days.

#### Assessments
* code_module **(categorical)** – identification code of the module, to which the assessment belongs.
* code_presentation **(categorical)** - identification code of the presentation, to which the assessment belongs.
* id_assessment **(numeric)** – identification number of the assessment.
* assessment_type **(categorical)** – type of assessment. Three types of assessments exist: Tutor Marked Assessment (TMA), Computer Marked Assessment (CMA) and Final Exam (Exam).
* date **(numeric)** – information about the final submission date of the assessment calculated as the number of days since the start of the module-presentation. The starting date of the presentation has number 0 (zero).
* weight **(numeric)** - weight of the assessment in %. Typically, Exams are treated separately and have the weight 100%; the sum of all other assessments is 100%.

#### VLE
* id_site **(numeric)** – an identification number of the material.
* code_module **(categorical)** – an identification code for module.
* code_presentation **(categorical)** - the identification code of presentation.
* activity_type **(categorical)** – the role associated with the module material.
* week_from **(numeric)** – the week from which the material is planned to be used.
* week_to **(numeric)** – week until which the material is planned to be used.;

#### StudentInfo
* code_module **(categorical)** – an identification code for a module on which the student is registered.
* code_presentation **(categorical)** - the identification code of the presentation during which the student is registered on the module.
* id_student **(numeric)** – a unique identification number for the student.
* gender **(categorical)** – the student’s gender.
* region **(categorical)** – identifies the geographic region, where the student lived while taking the module-presentation.
* highest_education **(categorical)** – highest student education level on entry to the module presentation.
* imd_band **(categorical)** – specifies the Index of Multiple Depravation band of the place where the student lived during the module-presentation.
* age_band **(numeric)** – band of the student’s age.
* num_of_prev_attempts **(numeric)** – the number times the student has attempted this module.
* studied_credits **(numeric)** – the total number of credits for the modules the student is currently studying.
* disability **(categorical)**  – indicates whether the student has declared a disability.
* final_result **(categorical)** – student’s final result in the module-presentation.

#### StudentRegistragion
* code_module **(categorical)** – an identification code for a module.
* code_presentation **(categorical)** - the identification code of the presentation.
* id_student **(numeric)** – a unique identification number for the student.
* date_registration **(numeric)** – the date of student’s registration on the module presentation, this is the number of days measured relative to the start of the module-presentation (e.g. the negative value -30 means that the student registered to module presentation 30 days before it started).
* date_unregistration **(numeric)** – date of student unregistration from the module presentation, this is the number of days measured relative to the start of the module-presentation. Students, who completed the course have this field empty. Students who unregistered have Withdrawal as the value of the final_result column in the studentInfo.csv file.

#### StudentAssessment
* id_assessment **(numeric)** – the identification number of the assessment.
* id_student **(numeric)** – a unique identification number for the student.
* date_submitted **(numeric)** – the date of student submission, measured as the number of days since the start of the module presentation.
* is_banked **(numeric)** – a status flag indicating that the assessment result has been transferred from a previous presentation.
* score **(numeric)** – the student’s score in this assessment. The range is from 0 to 100. The score lower than 40 is interpreted as Fail. The marks are in the range from 0 to 100.

#### StudentVLE
* code_module **(categorical)** – an identification code for a module.
* code_presentation **(categorical)** - the identification code of the module presentation.
* id_student **(numeric)** – a unique identification number for the student.
* id_site **(numeric)** - an identification number for the VLE material.
* date **(numeric)** – the date of student’s interaction with the material measured as the number of days since the start of the module-presentation.
* sum_click **(numeric)** – the number of times a student interacts with the material in that day.



In [ ]:
vle.activity_type.value_counts()

### Data Description

 ## this is all stated in the cell above, keep here or above?
 
The dataset was originally used in a relational database schema with separated tables, hence the seven CSV files. In order to create a clustering model, we needed to merge these tables together into a singular dataframe. The resulting dataset had over **x million instances** encompassing data from 32,593 students and we found that our machines were bogged down by the size, making it necessary to take a random sample of 1,000 students for a total of 2,705,197 instances. 


 **below is the stuff that seems redundant (added some of it in the above paragraph too**
 - The data set was collected from students living in the UK
 - VLE stands for Virtual Learning Environment
 - VLE table records interactions with VLE 
 - 32,593 students
 - 22 courses
  
  
  we found that using all of the 32000 students was too intensive, so we sampled 4000 random students to create the dataframe from

In [ ]:
%%time
# Randomly sampling 4,000 records StudentVle Table with Million Rows.
studentSample = studentInfo.sample(n=4000, random_state = 777)
print(studentSample.shape)

# vle
# assessments X
# courses X
# studentAssessments X
# studentInfo X
# studentRegistration X
# studentVleSample X

df = pd.merge(studentSample, studentVle, on=['code_module', 'code_presentation', 'id_student'])
df2 = pd.merge(studentAssessments, assessments, on='id_assessment')
df = pd.merge(df, df2, on=['code_module', 'code_presentation', 'id_student'], how='left')
df = pd.merge(df, courses, on= ['code_module', 'code_presentation'], how='left')
df = pd.merge(df, studentRegistration, on=['code_module', 'code_presentation', 'id_student'], how='left')
df = pd.merge(df, vle, on=['id_site', 'code_module', 'code_presentation'], how='left')
print(df.shape)
df.head()

In [ ]:
The code below was used to ensure that we did not create any rows with missing values.

### Missing Values

In [ ]:
studentVle.head()

From the cell below, we saw that date_unregistration had null entries in over 96% of the data, and both week_from and week_to had null entries for over 86% of the data. Since these columns seemed to be unnecessary, we chose to drop all three as they consisted mostly of missing values. These columns could be used to analyze the population of the data that are not registered. **I don't understand this last sentence**

Running the same code on the dataset after dropping the three columns showed that the percentages of missing values in the remaining columns were less than 5%.

In [ ]:
print('Missing values: \n', df.isnull().sum() * 100 / len(df),'\n\n')
df = df.drop(columns = ['week_from', 'week_to', 'date_unregistration'])
print('Missing values after dopping columns: \n', df.isnull().sum() * 100 / len(df))

From the code below, we can see that the missing values within the IMD Band variable come from the Northern Region and Ireland. The next step was to choose how best to deal with the missing values within the dataset.

In [ ]:
missingIMDBand = df[df['imd_band'].isnull()]
print(missingIMDBand.region.value_counts() * 100 / len(missingIMDBand))

print('counts per region: \n',missingIMDBand.region.value_counts(),'\n\n')
print('percentage breakdown per region: \n',missingIMDBand.region.value_counts() * 100 / len(missingIMDBand), '\n\n')

northRegionMV = missingIMDBand.region.value_counts()[0]
irelandMV = missingIMDBand.region.value_counts()[1]
westMidRegionMV = missingIMDBand.region.value_counts()[2]
southRegionMV = missingIMDBand.region.value_counts()[3]

print('North Region percent of total: ',northRegionMV/len(df.region[df.region=='North Region']))
print('Ireland percent of total: ',irelandMV/len(df.region[df.region=='Ireland']))
print('West Midlands Region percent of total: ',westMidRegionMV/len(df.region[df.region=='West Midlands Region']))
print('South Region percent of total: ',southRegionMV/len(df.region[df.region=='South Region']))

In [ ]:
print(df.imd_band.value_counts(),"\n\n")

print(df[(df.region=='North Region') | (df.region=='Ireland')].imd_band.value_counts())

If we were particularly interested in examining regional differences, we theorized that we could calculate the most likely IMD_band value per region and fill in the missing values to match that distribution. However since the missing values only accounted for less than 8% of the total data, we chose to drop all the rows containing missing values, as data size requirements were still more than satisfied.

In [ ]:
df.dropna(inplace=True)
df.drop(columns=['id_student','id_assessment','id_site','code_module','code_presentation'],inplace=True)
df.shape


### One Hot Encoding

As shown in the code output below, many of the variables in the OULAD were categorical making it necessary to One Hot Encode the data in order to analyze it.

In [ ]:
df.dtypes

In [ ]:
# View unique values in each column
# for col in df:
#   print(df[col].unique())

In [ ]:
#Creating variables for indexing continuous and categorical variables
conCol = ['id_student', 'id_site', 'date_x', 'sum_click', 'num_of_prev_attempts', 
          'studied_credits', 'id_assessment ', 'date_submitted', 'is_banked', 'score', 'date_y', 'weight']

catCol = ['code_module', 'code_presentation', 'gender', 'region', 'highest_education', 
          'imd_band', 'age_band', 'disability', 'final_result', 'assessment_type' ]

In [ ]:
%%time
#Declaring Scalar object
scl_obj = StandardScaler()

#One hot encoding of ALL categorical variables
OneHotDF = pd.get_dummies(df[catCol],drop_first=False)

#Scaling non-encoded data
conVar = df.select_dtypes(exclude='object')
colnames = pd.Series(conVar.columns)
conVarScaled = scl_obj.fit_transform(conVar)
conVarScaled = pd.DataFrame(data = conVarScaled, columns= colnames)

#Combining with continuous variables from cleaned dataset
OneHotDF = OneHotDF.reset_index()
OneHotDF.pop('index')
OneHotDF = pd.concat([conVarScaled,OneHotDF], axis = 1)
OneHotDF.head()

#https://github.com/jakemdrew/DataMiningNotebooks/blob/master/01.%20Pandas.ipynb


### Outliers

Once the OULAD was entirely numeric, we could look through the data for outliers. 

In [ ]:
Q1 = OneHotDF.quantile(0.25)
Q3 = OneHotDF.quantile(0.75)
IQR = Q3 - Q1
print(IQR)

# Data Understanding 2 #
*Visualize the any important attributes appropriately. Important: Provide an interpretation for any charts or graphs.*

# Modeling and Evaluation 1 #
*Train and adjust parameters*

# Modeling and Evaluation 2	#
*Evaluate and Compare*

In [ ]:
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier

y = OneHotDF['final_result_Pass']
X = OneHotDF[['num_of_prev_attempts','studied_credits','disability_N']]
cv = StratifiedKFold(n_splits=10)
# run at n =100
clf = RandomForestClassifier(n_estimators=2,random_state=1)

acc = cross_val_score(clf,X,y=y,cv=cv)

print ("Average accuracy = ", acc.mean()*100, "+-", acc.std()*100)

In [ ]:
OneHotDF.columns


In [ ]:
OneHotDFDropNA = OneHotDF.dropna()

In [ ]:
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd

X1 = OneHotDFDropNA[['num_of_prev_attempts','final_result_Pass']]

cls = KMeans(n_clusters=2, init='k-means++',random_state=1)
cls.fit(X1)
newfeature = cls.labels_ # the labels from kmeans clustering

# y = OneHotDF['Survived']
# X = OneHotDF[['Age','IsMale','Parch','SibSp']]
# X = np.column_stack((X,pd.get_dummies(newfeature)))

# acc = cross_val_score(clf,X,y=y,cv=cv)

# print ("Average accuracy (with kmeans for class/fare)= ", acc.mean()*100, "+-", acc.std()*100)

# Modeling and Evaluation 3 #
*Visualize Results*

# Modeling and Evaluation 4 #
*Summarize the Ramifications*

# Deployment #
*Be critical of your performance and tell the reader how you current model might be usable by other parties. Did you achieve your goals? If not, can you reign in the utility of your modeling? How useful is your model for interested parties (i.e., the companies or organizations that might want to use it)? How would your deploy your model for interested parties? What other data should be collected? How often would the model need to be updated, etc.?*

# Exceptional Work #